In [64]:
%reload_ext autoreload
%autoreload 2
import os,sys
sys.path.append('..')
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from BNN_SVI import BNN_SVI
import matplotlib.pyplot as plt
import pickle

In [65]:
 def get_data(dataset, split_id):
    ds_prefix = '../UCI_Datasets/'
    data = np.loadtxt('../UCI_Datasets/' + dataset + "/data/data.txt")
    xs = data[:,:-1]
    ys = data[:,-1]
    n_splits = np.loadtxt('../UCI_Datasets/' + dataset + "/data/n_splits.txt", dtype = np.int64)
    if split_id >= n_splits:
        train_x = None
        train_y = None
        test_x  = None
        test_y = None
        tau = None
        n_hidden = None
        n_epochs = None
    else:
        train_id = np.loadtxt('../UCI_Datasets/' + dataset + "/data/index_train_" + str(split_id) + ".txt", dtype = np.int64)
        test_id = np.loadtxt('../UCI_Datasets/' + dataset + "/data/index_test_" + str(split_id) + ".txt", dtype = np.int64)
        tau      = np.loadtxt('../UCI_Datasets/' + dataset + "/results/test_tau_100_xepochs_1_hidden_layers.txt")[split_id]
        n_hidden = np.loadtxt('../UCI_Datasets/' + dataset + "/data/n_hidden.txt", dtype = np.int64)
        n_epochs = np.loadtxt('../UCI_Datasets/' + dataset + "/data/n_epochs.txt", dtype = np.int64)

        train_x = xs[train_id]
        train_y = ys[train_id]
        test_x = xs[test_id]
        test_y = ys[test_id]
    return train_x, train_y, test_x, test_y,tau,n_hidden, n_splits, n_epochs

In [71]:
 def uci(dataset, split_id):
    train_x, train_y, test_x, test_y,tau, n_hiddens, n_splits, n_epochs = get_data(dataset, split_id)
    if(split_id >= n_splits):
        print("Invalid split_id")
        return np.nan, np.nan, np.nan, np.nan
    print('Dataset %s, split: %d' % (dataset, split_id))
    print('Model precision: %g' % tau)
    print('n_hiddens:%d' % n_hiddens)

    conf = dict()
    conf['num_iters'] = 50*n_epochs * (1+int(train_x.shape[0] / 128))    
    conf['noise_level'] = 1/np.sqrt(tau)
    model = BNN_SVI(train_x.shape[1], num_hiddens = [n_hiddens], conf = conf)
    model.train(torch.FloatTensor(train_x), torch.FloatTensor(train_y))
    rmse, nll_gaussian,nll = model.validate(torch.FloatTensor(test_x), torch.FloatTensor(test_y), num_samples=1000)
    print('RMSE = %g, NLL_gaussian = %6.3f, NLL = %6.3f' % (rmse, nll_gaussian, nll))
    return rmse, nll_gaussian, nll

In [ ]:
ds = [
    'bostonHousing'
, 'concrete'
, 'energy'
, 'kin8nm'
, 'naval-propulsion-plant'
, 'power-plant'
, 'protein-tertiary-structure'
, 'wine-quality-red'
, 'yacht'

]
stat = dict()
for d in ds:
    stat[d] = []
    for split_id in range(20):
        stat[d].append(uci(d,split_id))   
        f = open("stat_pyro_svi.pkl","wb")
        pickle.dump(stat,f)
        f.close()